In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
class DNAGCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(DNAGCN, self).__init__()
        # Graph convolutional layers
        self.conv_layers = nn.ModuleList()
        self.conv_layers.append(GCNConv(input_dim, hidden_dim))
        for _ in range(num_layers - 2):
            self.conv_layers.append(GCNConv(hidden_dim, hidden_dim))
        self.conv_layers.append(GCNConv(hidden_dim, output_dim))
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Apply graph convolutional layers
        for conv_layer in self.conv_layers:
            x = F.relu(conv_layer(x, edge_index))
        return x
# Example usage:
# Create a random DNA sequence bond matrix (adjacency matrix)
# For simplicity, assume a sequence length of 10
input_dim = 4  # For example, if DNA sequences are represented using one-hot encoding
hidden_dim = 64
output_dim = 128
num_layers = 3

sequence_length = 10
adjacency_matrix = torch.rand((sequence_length, sequence_length))
# Convert the adjacency matrix to edge indices
edges = torch.nonzero(adjacency_matrix, as_tuple=True)
# Create a simple data object with ‘x’ as node features and ‘edge_index’ as edge indices
data = torch_geometric.data.Data(x=torch.randn(sequence_length, input_dim), edge_index=edges)
# Instantiate and forward pass through the DNAGCN model

model = DNAGCN(input_dim, hidden_dim, output_dim, num_layers)
output = model(data)

ValueError: `MessagePassing.propagate` only supports integer tensors of shape `[2, num_messages]`, `torch_sparse.SparseTensor` or `torch.sparse.Tensor` for argument `edge_index`.

In [5]:
edges

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7,
         7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9]),
 tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3,
         4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7,
         8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
         2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
         6, 7, 8, 9]))